In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [3]:
data = pd.read_csv('D:\DOCUMENT\INFORMATIKA\FULL PROJEK\FastAPI\Wine value classification\winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
data.info()
data.describe().T.style.background_gradient(axis=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.000000,8.319637,1.741096,4.600000,7.100000,7.900000,9.200000,15.900000
volatile acidity,1599.000000,0.527821,0.179060,0.120000,0.390000,0.520000,0.640000,1.580000
citric acid,1599.000000,0.270976,0.194801,0.000000,0.090000,0.260000,0.420000,1.000000
residual sugar,1599.000000,2.538806,1.409928,0.900000,1.900000,2.200000,2.600000,15.500000
chlorides,1599.000000,0.087467,0.047065,0.012000,0.070000,0.079000,0.090000,0.611000
free sulfur dioxide,1599.000000,15.874922,10.460157,1.000000,7.000000,14.000000,21.000000,72.000000
total sulfur dioxide,1599.000000,46.467792,32.895324,6.000000,22.000000,38.000000,62.000000,289.000000
density,1599.000000,0.996747,0.001887,0.990070,0.995600,0.996750,0.997835,1.003690
pH,1599.000000,3.311113,0.154386,2.740000,3.210000,3.310000,3.400000,4.010000
sulphates,1599.000000,0.658149,0.169507,0.330000,0.550000,0.620000,0.730000,2.000000


In [6]:
data = data.replace({'quality': {8 : 'Good', 7: 'Good', 6: 'Average', 5:'Average', 4:'Bad', 3:'Bad'}})
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Avarage
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Avarage
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Avarage
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Avarage
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Avarage


In [8]:
X = data.drop(columns='quality')
y = data.quality

X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:

#SCALING
scaler = MinMaxScaler(feature_range=(0,1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
models = {
    'LogisticRegression' : LogisticRegression(),
    'SVM' : SVC(),
    'Random Forest' : RandomForestClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'KNeighbors' : KNeighborsClassifier(),
    'Naive Bayes' : GaussianNB()
}

In [19]:
result = {}
for model_name, model in models.items():
    print(f'training {model_name}...')
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    result[model_name] = accuracy
    print(f'{model_name}, Accuracy: {accuracy}\n')

training LogisticRegression...
LogisticRegression, Accuracy: 0.828125

training SVM...
SVM, Accuracy: 0.8375

training Random Forest...
Random Forest, Accuracy: 0.871875

training Decision Tree...
Decision Tree, Accuracy: 0.79375

training KNeighbors...
KNeighbors, Accuracy: 0.840625

training Naive Bayes...
Naive Bayes, Accuracy: 0.803125



In [20]:
sorted_result = sorted(result.items(), key=lambda x: x[1], reverse=True)
print('\nModel Comparison Accuracy: ')

for model_name, accuracy in sorted_result:
    print(f'{model_name}: {accuracy:.4f}')


Model Comparison Accuracy: 
Random Forest: 0.8719
KNeighbors: 0.8406
SVM: 0.8375
LogisticRegression: 0.8281
Naive Bayes: 0.8031
Decision Tree: 0.7937


In [23]:
#Hyperparameter tunning
param_grid = {
    'n_estimators' : [50, 100, 200],
    'max_depth': [10,20,30],
    'min_samples_split': [2,5,10]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)
print(f'best parameter for random forest: {grid_search.best_params_}')
print(f'best score for random forest: {grid_search.best_score_}')

Fitting 10 folds for each of 27 candidates, totalling 270 fits
best parameter for random forest: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 200}
best score for random forest: 0.8749200295275591


In [25]:
#cross validation score for the best model random forest
best_model = grid_search.best_estimator_
cv_scores= cross_val_score(best_model, X, y, cv=10)
print(f'Cross validation score for random forest: {cv_scores}')
print(f'Mean Cross validation score: {cv_scores.mean()}')

Cross validation score for random forest: [0.825      0.85625    0.84375    0.80625    0.8625     0.85
 0.79375    0.85625    0.83125    0.85534591]
Mean Cross validation score: 0.8380345911949686


In [26]:
from sklearn.metrics import classification_report

y_pred = best_model.predict(X_test_scaled)
print('\nClassification Report: ')
print(classification_report(y_test, y_pred, zero_division=1))


Classification Report: 
              precision    recall  f1-score   support

     Avarage       0.88      0.97      0.92       262
         Bad       1.00      0.00      0.00        11
        Good       0.73      0.51      0.60        47

    accuracy                           0.87       320
   macro avg       0.87      0.49      0.51       320
weighted avg       0.86      0.87      0.84       320



In [28]:
import joblib

#save best model
joblib.dump(best_model, 'best_model.pkl')

#save standart scaler
joblib.dump(scaler, 'scaler.pkl')

print('model and scaler have ben saved')

model and scaler have ben saved
